In [1]:
#import 相關的包和mnist資料集
import numpy as np
import matplotlib.pyplot as plt
from time import time
from sklearn import datasets, manifold

#定義函式將結果plot出來                            
def plot_embedding(X, title=None):     
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)                           

    plt.figure()              
    ax = plt.subplot(111)      
    for i in range(X.shape[0]):                          
        plt.text(X[i, 0], X[i, 1], str(digits.target[i]),    
                 color=plt.cm.Set1(y[i] / 10.),         
                 fontdict={'weight': 'bold', 'size': 9})
    plt.xticks([]), plt.yticks([])
    if title is not None:
        plt.title(title)

# 呼叫t-SNE
print("Computing t-SNE embedding")           
tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
t0 = time()
X_tsne = tsne.fit_transform(X)

plot_embedding(X_tsne,                                   
            "t-SNE embedding of the digits (time %.2fs)" %
            (time() - t0))                           

plt.show() 

Computing t-SNE embedding


NameError: name 'X' is not defined

In [5]:
import numpy as np
from sklearn.manifold import TSNE
X = np.array([[0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 1]])
X_embedded = TSNE(n_components=2).fit_transform(X)
X_embedded.shape
# X_embedded

(4, 2)

In [21]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib.patches as mpatches
import time

# Classifier Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import collections


# Other Libraries
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold
import warnings

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, RobustScaler

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model  import LogisticRegression

In [22]:
dt = pd.read_csv('../train.csv' , encoding='big5')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (17,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
def data_cl(dt):
    dt = dt.drop("CUS_ID" ,axis=1)
    
    ohi = pd.get_dummies(dt["GENDER"])
    dt = dt.drop("GENDER",axis=1)
    dt = pd.concat([ohi,dt],axis=1)
    
    ohi = pd.get_dummies(dt["AGE"])
    dt = dt.drop("AGE",axis=1)
    dt = pd.concat([ohi,dt],axis=1)

    labelencoder = LabelEncoder()
    for i in dt.keys():    
        for a in dt[i]:
            if  a is not None and isinstance(a,str):
                dt[i] = labelencoder.fit_transform(dt[i].astype(str))
                break
    
    for i in dt.keys():
        dt[i] = dt[i].fillna(dt[i].mean())
    
    return dt    
    

In [24]:
dt = data_cl(dt)
dt.shape

(100000, 135)

In [25]:
def logreg(dt):
    x = dt.drop("Y1",axis=1)
    y = dt["Y1"]
    train_X, test_X, train_y, test_y = train_test_split(x, y, test_size = 0.3)

    lr=LogisticRegression()
    lr.fit(train_X,train_y)


    # 印出截距
#     print(lr.intercept_)
    # 印出係數
    a = abs(lr.coef_)[0]
    c =lr.coef_[0]
    a = list(a)
    c = list(c)
    newa = list(c)
    newa.sort()
    x = []
    for i in newa:
        if abs(i) < 0.01:
            x.append(dt.columns[c.index(i)])
    return x

In [26]:
x = logreg(dt)
dt_fix1 = dt.drop(x,axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [27]:
X_embedded = TSNE(n_components=2).fit_transform(dt_fix1)

In [34]:
X_embedded[-1]

array([-15.915314, -27.220669], dtype=float32)

In [35]:
from sklearn import  ensemble, preprocessing, metrics
from sklearn.model_selection import train_test_split

xf = X_embedded
yf = dt_fix1["Y1"]
train_x, test_x, train_y, test_y = train_test_split(xf, yf, test_size = 0.3)

forest = ensemble.RandomForestClassifier(n_estimators = 100)
forest_fit = forest.fit(train_x, train_y)

In [36]:
test_y_predicted = forest.predict(test_x)

# 績效
fpr, tpr, thresholds = metrics.roc_curve(test_y, test_y_predicted)
auc = metrics.auc(fpr, tpr)
print(auc)

0.5011422524985304


In [37]:
test_y_predicted = forest.predict(dt.drop("Y1",axis=1))

# 績效
fpr, tpr, thresholds = metrics.roc_curve(dt["Y1"], test_y_predicted)
auc = metrics.auc(fpr, tpr)
print(auc)

ValueError: Number of features of the model must match the input. Model n_features is 2 and input n_features is 134 

In [38]:
from sklearn.decomposition import PCA

In [50]:
xpca = PCA(dt_fix1.drop(dt_fix1.keys(),axis=1))

TypeError: 'PCA' object is not subscriptable

In [54]:
from sklearn import  ensemble, preprocessing, metrics
from sklearn.model_selection import train_test_split

x = xpca
y = dt_fix1["Y1"]
train_X, test_X, train_y, test_y = train_test_split(x, y, test_size = 0.33)

forest = ensemble.RandomForestClassifier(n_estimators = 300)
forest_fit = forest.fit(train_X, train_y)

TypeError: Singleton array array(PCA(copy=True, iterated_power='auto',
    n_components=Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]

[100000 rows x 0 columns],
    random_state=None, svd_solver='auto', tol=0.0, whiten=False),
      dtype=object) cannot be considered a valid collection.